# Analyzing Data for Rotational Churners

Loading the churner data and data of the newer/more recent users

In [1]:
import numpy as np
import pandas as pd

dfs = pd.ExcelFile('data/mock/sample_mock_rc_data.xlsx')
sheets = dfs.sheet_names
d = []
for i in range(10):
    d.append(dfs.parse(sheet_name=sheets[i]))

newer_users = pd.read_excel('data/mock/sample_mock_nu_data.xlsx')

In [2]:
d[0].head()

,weeks,IMEI,credits_loaded_per_week,num_hrs_spend_on_net,mobile_data_usage(gb),promos_used,active_times,activity_status
0,2019-07-17,823865145970266,100,5.3,8.07,0,1,0
1,2019-07-24,823865145970266,0,17.4,8.60,0,0,0
2,2019-07-31,823865145970266,200,15.5,5.23,2,1,0
3,2019-08-07,823865145970266,20,13.2,6.87,2,0,0
4,2019-08-14,823865145970266,0,2.7,5.59,3,0,0


In [3]:
newer_users.head()

,weeks,IMEI,credits_loaded_per_week,num_hrs_spend_on_net,mobile_data_usage(gb),promos_used,active_times,activity_status
0,2020-02-12,299580108038267,90,5.9,8.1,0,2,0
1,2020-02-19,299580108038267,30,17.4,11.1,0,0,0
2,2020-02-26,299580108038267,100,16.8,4.9,3,2,0
3,2020-03-04,299580108038267,0,12.4,7.5,1,0,0
4,2020-03-11,299580108038267,0,2.7,5.3,2,0,0


#### Using Euchlidean Distance to Compare how Similar 2 the New User is to a Churner

In [4]:
def Euclidean_Dist(df1, df2, cols=d[0].columns[2:-1]):
    return (np.linalg.norm(df1[cols].values - df2[cols].values, axis=1))

In [5]:
df1 = d[0].iloc[:-4, :-1]
df2 = newer_users.iloc[:len(df1), :-1]
df3 = d[1].iloc[:len(df1), :-1]
print(len(df1), ' ', len(df2), ' ', len(df3))

26   26   26


In [6]:
distances = Euclidean_Dist(df1, df2)
distances

array([ 10.06781506,  30.10398645, 100.0189927 ,  20.05085784,
         1.04120123,  20.09977363,  30.01670368,   2.91391489,
        40.00002   , 500.00676005,  20.08009213,   2.64577021,
        30.04399441,   2.47879003,  10.17150923,  30.04065412,
         3.68814316,  30.1662477 ,   3.24266865,  30.10967286,
       200.02877918,   2.98799264,  10.00008   ,   1.00079968,
         1.78538511,  30.16312484])

In [7]:
distances.mean()

44.72898959526685

In [8]:
# comparing it to another user who is not related
distances1 = Euclidean_Dist(df2, df3)
distances1

array([ 90.53333972,  23.83145191, 101.00390537,   8.99204649,
        31.26667395, 100.26185915,  60.2511253 ,  50.07073097,
        70.14693222, 300.02515161,  20.32950811,  52.08937032,
        50.50728264, 200.03752548, 101.01862452,   1.54064921,
       500.09171719,  40.21729603,  13.01892853,  20.52264359,
       300.54482611,  70.24791029,  20.15510109,   4.21966823,
       900.00231383,  30.24086804])

In [9]:
distances1.mean()

121.58336345770392

#### As we can see, the new user is much more similar to churner 1 compared to churner 2.

In [10]:
df1_sums = df1.groupby('IMEI')[d[0].columns[2:-1]].sum()
df2_sums = df2.groupby('IMEI')[d[0].columns[2:-1]].sum()

In [11]:
# Modifying the Euchlidean Distance function
def Mod_Euclidean_Dist(df1,df2,cols=d[0].columns[2:-1]):
    x = df1.groupby('IMEI')[d[0].columns[2:-1]].sum()
    y = df2.groupby('IMEI')[d[0].columns[2:-1]].sum()

    return np.linalg.norm(x.values - y.values,
                   axis=1)

In [12]:
mod_distances = Mod_Euclidean_Dist(df1,df2)
mod_distances 

# the first euclidean distance result produced a much better result in terms of comparing the difference of the 2 dataframes

array([600.02220092])

### Different Computation Approach for Comparing

Computation:
 - a = list of sums in each column of df1
 - b = list of sums in each column of df2
 - c = l/h (list of similarities per column; l = lower value in a certain index of lists a and b; h = higher value)

 similarity = (sum(c)/len(c))*100

In [13]:
percent_sim = []

for i in range(len(df1_sums.columns.tolist())):
    x = 0
    y = 0
    if df1_sums[df1_sums.columns.tolist()[i]].tolist() < df2_sums[df2_sums.columns.tolist()[i]].tolist():
        x = df1_sums[df1_sums.columns.tolist()[i]].tolist()[0]
        y = df2_sums[df2_sums.columns.tolist()[i]].tolist()[0]
    else:
        x = df2_sums[df2_sums.columns.tolist()[i]].tolist()[0]
        y = df1_sums[df1_sums.columns.tolist()[i]].tolist()[0]
    percent_sim.append(x/y)

In [14]:
df1_sums

,credits_loaded_per_week,num_hrs_spend_on_net,mobile_data_usage(gb),promos_used,active_times
IMEI,,,,,
823865145970266,2940,254.1,178.14,50,23


In [15]:
df2_sums

,credits_loaded_per_week,num_hrs_spend_on_net,mobile_data_usage(gb),promos_used,active_times
IMEI,,,,,
299580108038267,2340,253.9,183.1,51,22


In [16]:
ave_sim = sum(percent_sim)/len(percent_sim)*100
print('Similarity is: {:.2f}%'.format(float(ave_sim)))

Similarity is: 94.10%


### Computing Similarity per Row and Computing the Average Similarity

In [17]:
df1.head(3)

,weeks,IMEI,credits_loaded_per_week,num_hrs_spend_on_net,mobile_data_usage(gb),promos_used,active_times
0,2019-07-17,823865145970266,100,5.3,8.07,0,1
1,2019-07-24,823865145970266,0,17.4,8.60,0,0
2,2019-07-31,823865145970266,200,15.5,5.23,2,1


In [18]:
df2.head(3)

,weeks,IMEI,credits_loaded_per_week,num_hrs_spend_on_net,mobile_data_usage(gb),promos_used,active_times
0,2020-02-12,299580108038267,90,5.9,8.1,0,2
1,2020-02-19,299580108038267,30,17.4,11.1,0,0
2,2020-02-26,299580108038267,100,16.8,4.9,3,2


In [19]:
vals = [[],[],[],[],[]]

z=0
for i in df1.columns[2:]:
    for j in range(len(df1[i].tolist())):
        x = 0
        y = 0
        if df1[i].tolist()[j] < df2[i].tolist()[j]:
            x = df1[i].tolist()[j]
            y = df2[i].tolist()[j]
        else:
            y = df1[i].tolist()[j]
            x = df2[i].tolist()[j]
        
        if x == 0 and y == 0:
            vals[z].append(100.00)
        else:
            vals[z].append(round((x/y)*100,2))
    z+=1

# I could have just used df1['column'] / df2['column'] or df2['column'] / df1['column'], but it does not check for division by 0s.
# I also needed to check which value was lower as that will be the dividend.

In [20]:
data_dict = {}
k=0
for i in df1.columns[2:]:
    data_dict[i] = vals[k]
    k+=1

sim_df = pd.DataFrame(data_dict)

In [21]:
sim_df.head()

,credits_loaded_per_week,num_hrs_spend_on_net,mobile_data_usage(gb),promos_used,active_times
0,90.0,89.83,99.63,100.00,50.0
1,0.0,100.00,77.48,100.00,100.0
2,50.0,92.26,93.69,66.67,50.0
3,0.0,93.94,91.60,50.00,100.0
4,100.0,100.00,94.81,66.67,100.0


In [22]:
sim_df['similarity'] = ((sim_df[sim_df.columns[0]] + sim_df[sim_df.columns[1]] + sim_df[sim_df.columns[2]] + sim_df[sim_df.columns[3]] + sim_df[sim_df.columns[4]])/500)*100
sim_df.head()

,credits_loaded_per_week,num_hrs_spend_on_net,mobile_data_usage(gb),promos_used,active_times,similarity
0,90.0,89.83,99.63,100.00,50.0,85.892
1,0.0,100.00,77.48,100.00,100.0,75.496
2,50.0,92.26,93.69,66.67,50.0,70.524
3,0.0,93.94,91.60,50.00,100.0,67.108
4,100.0,100.00,94.81,66.67,100.0,92.296


In [23]:
sim_df['similarity'].mean()

72.87092307692308